In [54]:
import nltk
import gensim
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer
import string
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem.porter import *
import os
import numpy as np
import glob
from gensim import models
from gensim import similarities
#nltk.download('stopwords')

In [131]:
#Fetching all the text files

SIM = []
path = 'by_page_csv/'
folders = [f for f in glob.glob(path + "**/*.csv", recursive=True)] 
#folders = folders[20:30]

In [132]:
results = None
for f in folders:
    SIM.append([baseFunction(f)])

In [56]:
def baseFunction(f):
    sims = None
    perPageText = pd.read_csv(f, error_bad_lines=False)
    perPageText = perPageText.sort_values(['page'])
    
    tokens_ocr = fetch_ocr_tokens(perPageText.ocr)
    tokens_tcp = fetch_tcp_tokens(perPageText.tcp)
    
    tokens_tcp = remove_stop_words(tokens_tcp)
    tokens_ocr = remove_stop_words(tokens_ocr)
    
    obj = GetSimilarityIndex(tokens_tcp)
    sims = GerSimilarityLSI(tokens_ocr, obj[0], obj[1], obj[2], obj[3])
    #sims = GerSimilarity(tokens_ocr, obj[0], obj[1], obj[2])
        
    return sims

In [57]:
def fetch_ocr_tokens(textPage):
    tokens = []
    i = 0
    while i < len(textPage):
        lowers = textPage[i].lower()
        tokens.append(stem_tokens(nltk.RegexpTokenizer(r'[a-zA-Z]{2,}').tokenize(lowers), stemmer))
        i += 1
    return tokens

In [58]:
def fetch_tcp_tokens(textPage):
    tokens = []
    i = 0
    while i < len(textPage):
        lowers = textPage[i].lower()
        tokens.append(stem_tokens(nltk.RegexpTokenizer(r'\w+').tokenize(lowers), stemmer))
        i += 1
    return tokens

In [59]:
token_dict = {}
stemmer = PorterStemmer()

In [60]:
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

In [61]:
def remove_stop_words(tokens):
    filtered = [w for w in tokens if not w in stopwords.words('english')]
    return filtered

In [62]:
perPageText = pd.read_csv('by_page_csv/0000900501_by_page.csv', error_bad_lines=False)

In [119]:
def GetSimilarityIndex(tokens_tcp):

    sims = None
    #The ground truth (tcp)
    dictionary_tcp = gensim.corpora.Dictionary(tokens_tcp)
    #print(dictionary.token2id)
    corpus_tcp = [dictionary_tcp.doc2bow(token_tcp) for token_tcp in tokens_tcp]
    #print(corpus)
    tf_idf_tcp = gensim.models.TfidfModel(corpus_tcp)
    #building the index
    sims = gensim.similarities.MatrixSimilarity(tf_idf_tcp[corpus_tcp], num_features=len(dictionary_tcp))
    
    return [sims, dictionary_tcp, tf_idf_tcp, corpus_tcp]

In [21]:
# tf_idf calculated for each document and similarity is checked

def GerSimilarity(tokens_ocr, sims, dictionary_tcp, tf_idf_tcp):
    i = 0
    avg_sims = []
    while i < len(tokens_ocr):
        query_doc_bow = dictionary_tcp.doc2bow(tokens_ocr[i])
        query_doc_tf_idf = tf_idf_tcp[query_doc_bow]
        sum_of_sims =(np.sum(sims[query_doc_tf_idf], dtype = np.float32))
        avg = sum_of_sims / len(tokens_ocr[i]) * 100
        avg_sims.append(avg) 
        i += 1

    if (len(avg_sims) == len(tokens_ocr)):
        total_avg = np.sum(avg_sims, dtype = np.float)
        percentage_of_similarity = total_avg / len(tokens_ocr)
        
    return percentage_of_similarity

In [130]:
def GerSimilarityLSI(tokens_ocr, sims, dictionary_tcp, tf_idf_tcp, corpus):
    lsi = models.LsiModel(corpus, id2word=dictionary_tcp, num_topics=10)
    index = similarities.MatrixSimilarity(lsi[corpus])
    index.save('/tmp/deerwester.index')
    index = similarities.MatrixSimilarity.load('/tmp/deerwester.index')
    i = 0
    avg_sims = []
    while i < len(tokens_ocr):
        vec_bow = dictionary_tcp.doc2bow(tokens_ocr[i])
        vec_lsi = lsi[vec_bow]
        sims = index[vec_lsi]
        sum_of_sims =(np.sum(sims, dtype = np.float32))
        avg = sum_of_sims / len(sims) * 100
        avg_sims.append(avg)
        i += 1
    
    total_avg = np.sum(avg_sims, dtype = np.float) / len(avg_sims)
    
    return total_avg

In [133]:
import pickle

In [138]:
#Write
with open('SIM', 'wb') as fp:
    pickle.dump(SIM, fp)

In [139]:
#Read
with open ('SIM', 'rb') as fp:
    similarities = pickle.load(fp)

In [141]:
len(similarities)

2346

In [142]:
similarities

[[67.51643384262465],
 [70.95791023064092],
 [70.83261171272257],
 [100.0],
 [80.69294628740647],
 [83.22331329927584],
 [86.36694319592041],
 [83.88143549025853],
 [78.97776338544],
 [81.57493472099304],
 [60.151215841078894],
 [77.9343499802053],
 [71.45603319560658],
 [76.42972736185442],
 [88.71674773559918],
 [60.41368203755667],
 [81.09890771503292],
 [85.5673967733891],
 [75.0049618106258],
 [76.7237230407433],
 [79.61635403173929],
 [72.96924949959411],
 [67.58242392585048],
 [88.0933789147271],
 [74.29447669733189],
 [74.69775332076999],
 [82.43030919623146],
 [100.0],
 [61.44516744415653],
 [82.7825857091833],
 [65.77283902539497],
 [81.31375788133165],
 [80.68352871470982],
 [72.81501156588395],
 [83.4106964364937],
 [86.91690369229235],
 [73.28891777186003],
 [73.57401581112885],
 [70.08347651855748],
 [39.849412858486176],
 [38.56448154060208],
 [78.59178689809946],
 [48.48277664328739],
 [43.97243526246813],
 [84.03531520361683],
 [83.54091023886897],
 [69.7086098042785],